In [2]:
!python -m spacy download en_core_web_md

2023-08-12 10:47:18.364056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 25.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [5]:
import spacy
import os
import glob
import re
import string
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import pickle
nlp = spacy.load('en_core_web_md')

In [7]:
def clean(text):
  '''
  A function to clean the text.

  Input:
  text: string
  '''
  # lowercase the text
  text = text.lower()
  # remove unicode characters
  text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)

  return text

In [8]:
def getAdj(text,index):

  '''
  This is a fuctionn that checks if there are any adjectives associated with subjects
  or objects extrated.

  Input:
  text: (string)
  index: (int)
  '''

  doc = nlp(text)

  phrase = ''
  pos = None

  for token in doc:

    if token.i == index:

      for subtoken in token.children:
        if (subtoken.pos_ == 'ADJ'):
          phrase += ' ' + subtoken.text
      break

  return phrase

def getSVO(text):
  '''
  This is the function that extracts the subject and object from a text and
  returns subject, object and verb (reflects relation between sub and obj).

  Input:
  text: (string)
  '''

  doc = nlp(text)
  sub = None
  rel = None
  obj = None

  for token in doc:
    # root word
    if (token.pos_=='VERB'):

        # only extract noun or pronoun subjects
      for sub_tok in token.lefts:

        if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):

          # look for subject modifier
          adj = getAdj(text,sub_tok.i)


          sub = (adj + ' ' + sub_tok.text).strip()

          rel = token.text

          # check for noun or pronoun direct objects
          for sub_tok in token.rights:

            if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):

              # look for object modifier
              adj = getAdj(text,sub_tok.i)

              obj = (adj + ' ' + sub_tok.text).strip()


  return sub, obj, rel

In [86]:
def getAnswer(text):
  '''
  A function that checks for the nodes and edge values to return answer to given
  query.

  Input:
  text: string
  '''
  cleaned_text = clean(text)

  sub, obj, verb = getSVO(cleaned_text)
  ans = None

  prob_answers = [k for k,v in edge_labels.items() if v == verb]
  if len(prob_answers) == 0:
    print('Answer not found')

  elif sub is None and obj is None:
    print('Answer not found')

  else:

    for prob_ans in prob_answers:
      if sub is not None:
        if (any(sub in i for i in prob_ans)):
          ans = prob_ans[1]
          break
      elif obj is not None:
        if (any(obj in i for i in prob_ans)):
          ans = prob_ans[0]
          break

    if ans is None:
      print('Answer not found.')
    else:
      print(ans)

#### Loading the Graph and its data

In [72]:
graph_path = '/content/drive/MyDrive/Intern Task/Graph_DB/graph_db.pickle'
G = pickle.load(open(graph_path, 'rb'))
edge_labels = nx.get_edge_attributes(G, 'value')

In [83]:
text = 'what hill has'

getAnswer(text)

unique charm


In [85]:
text = 'what do you like?'

getAnswer(text)

radiant complexion
